# First strategy

Below are basics steps that might be included in each strategy. 

### 1. Preparations
At first one needs to prepare the workspace - load data and libraries

In [1]:
# Import basic libraries.
import xarray as xr
import numpy as np
import pandas as pd

# Import quantnet libraries.
import qnt.data    as qndata
import qnt.stepper as qnstepper
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import qnt.ta      as qnta # indicators library
import qnt.forward_looking as qnfl # forward looking checking

import datetime as dt

In [2]:
# load historical data
data = qndata.load_data(
                       tail = dt.timedelta(days=4*365),
                       dims = ("time", "field", "asset"),
                       forward_order=True)

fetched chunk 1/5 2s
fetched chunk 2/5 4s
fetched chunk 3/5 5s
fetched chunk 4/5 7s
fetched chunk 5/5 8s
Data loaded 8s


"data" is xarray.DataArray that contains stocks historical data. For instance, we want Apple stock open and close prices:

In [3]:
apple_close = data.loc[::, "close", "NASDAQ:AAPL"]
apple_open = data.loc[::, "open", "NASDAQ:AAPL"]

display(apple_close.to_pandas()) # jupyter has a nice formatter for pandas series and dataframes

time
2016-06-06    2761.64
2016-06-07    2772.84
2016-06-08    2770.32
2016-06-09    2790.20
2016-06-10    2767.24
               ...   
2020-05-13    8614.20
2020-05-14    8667.12
2020-05-15    8615.88
2020-05-18        NaN
2020-05-19        NaN
Length: 996, dtype: float64

In [4]:
# you can also work with pandas:
# apple_close = data.loc[::, "close", :].to_pandas()["NASDAQ:AAPL"]

Available data explanation is [here](https://quantnet.ai/learn/:availableData_Table). Some other data:

In [5]:
all_close = data.loc[::, "close", :]
all_open = data.loc[::, "open", :]

# boolean parameter. True if the stock is in top 500 most liquid stocks over the last month
liquid = data.loc[::, "is_liquid", :]

### 2. Weights allocation

Suppose, we have a traiding idea - invest more if open price is low. This hypothesis can be expressed through the formula:

$$\frac{1}{open}$$

We can allocate capital by assigning weights to the portfolio instuments (more [details](https://quantnet.ai/learn/:gettingStarted)):

In [6]:
changes = qnta.change(all_close, 122)
weights = xr.where(changes > 0, 1, 0) # buy when price goes up, sell when price goes down

You can implement and test any idea you want. Some other examples:

In [7]:
# buy all positions: weights = all_open/all_open
# sell all positions: weights = -all_open/all_open
# the more price change, the more we buy = (all_close - all_open)/all_open

Notice that we trade only liquid stocks. One can form output weights:

In [8]:
output = weights*liquid

In [9]:
# If you worked with pandas and weigths is pandas.Dataframe:
# output = xr.DataArray(weights.values, dims = ["time","asset"], coords= {"time":weights.index,"asset":weights.columns} )

Output normalization, weights sum for one day should be <= 1

In [10]:
output = output / abs(output).sum('asset')

### 3. Perfomance estimation

Once we have constructed an algorithm we need to evaluate it. At first, we need to calculate statistic.

In [11]:
# calc_stat calculate statistic on a relevenat time frame window
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-05-13,1.156819,-0.004053,0.193347,-0.185435,-0.33275,0.126673,0.024492,1.0,854.0,0.130487,16.392030
2020-05-14,1.158645,0.001578,0.193345,-0.184149,-0.33275,0.132792,0.025675,1.0,854.0,0.130533,16.366131
2020-05-15,1.167786,0.007889,0.193339,-0.177712,-0.33275,0.132028,0.025526,1.0,854.0,0.130435,16.368478
2020-05-18,1.172441,0.003986,0.193351,-0.174435,-0.33275,0.138646,0.026807,1.0,854.0,0.130381,16.339192
2020-05-19,1.170744,-0.001447,0.192934,-0.175629,-0.33275,0.175693,0.033897,1.0,854.0,0.130373,16.248194


Algorithm results, calculated on historical data, are usually presented on an [equity graph](https://quantnet.ai/learn/:relativeReturns) in order to understand the behavior of the cumulative profit:

In [12]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

We use a set of [criteria](https://quantnet.ai/learn/:quality_criteria) to evaluate the performance. You can submit your algorithm and take part in a competition if it passes all the [requirements](https://quantnet.ai/learn/:requirements).

In [13]:
# Actual sharpe on a relevant timeframe. 
# According to the rules the Sharpe ratio must be greater than 1:
display(stat[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())

time,2020-05-19
field,
sharpe_ratio,0.175693


In [14]:
# According to the rules the correlation with other strategies must be less than 90%:
qnstats.print_correlation(output, data)


WARNING! This strategy correlates with other strategies.
The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 7
The max correlation value (with systems with a larger Sharpe ratio): 0.9999999999999999
Current sharpe ratio(3y): 0.17533273455107462



### 4. Submit

If you are satisfied enough with your algorithm and it passes all the requirements you can submit it.

In [15]:
# Finally, write the output
qnstepper.write_output(output)

write output: /root/fractions.nc.gz


At this stage the code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!